In [1]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
from EoM import *
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import math
import intelligent_robotics as ir

In [2]:
def rot_x(theta):
    rot_x = sp.Matrix([[1,0,0],
                       [0,sp.cos(theta),-sp.sin(theta)],
                       [0,sp.sin(theta),sp.cos(theta)]])
    return rot_x

def rot_y(theta):
    rot_y = sp.Matrix([[sp.cos(theta),0,sp.sin(theta)],
                       [0,1,0],
                       [-sp.sin(theta),0,sp.cos(theta)]])
    return rot_y

def rot_z(theta):
    rot_z = sp.Matrix([[sp.cos(theta),-sp.sin(theta),0],
                       [sp.sin(theta),sp.cos(theta),0],
                       [0,0,1]])
    return rot_z

def trans(a,b,c):
    x = sp.Matrix([[a],[b],[c]])
    return x

def get_torque_from_L(L,q,qd):
    round_L_round_q = sp.zeros(len(q),1);
    i = 0;
    for q_i in q:
        round_L_round_q_i = [];
        round_L_round_q_i = sp.diff(L,q_i);
        round_L_round_q[i] = round_L_round_q_i;
        i+=1;
     
    d_dt_round_L_round_qd = sp.zeros(len(qd),1);
    i = 0;
    for qd_i in qd:
        round_L_round_qd_i = [];
        d_dt_round_L_round_qd_i = [];
        round_L_round_qd_i = sp.diff(L,qd_i);
        d_dt_round_L_round_qd_i = time_derivative(round_L_round_qd_i,N);
        d_dt_round_L_round_qd[i] = d_dt_round_L_round_qd_i;
        i+=1;
        
    tau = d_dt_round_L_round_qd - round_L_round_q 
    return tau

In [3]:
r, l, D = sp.symbols('r, l, D')
m_w, m_b, g = sp.symbols('m_w, m_b, g')
I_w, I_b = sp.symbols('I_w, I_b')
theta_P, theta_w1, theta_w2 = dynamicsymbols('theta_P, theta_w1, theta_w2')

# Displacement Vector
x_c1 = trans(r*theta_w1,0,0)
x_w1 = x_c1 +  trans(0,D,r)
x_c2 = trans(r*theta_w2,0,0)
x_w2 = x_c2 +  trans(0,-D,r)
x_b = 0.5*(x_w1 + x_w2) +  rot_y(theta_P) @ trans(0,0,l)


# Velocity Vector
v_w1 = sp.simplify(time_derivative(x_w1,N))
v_w2 = sp.simplify(time_derivative(x_w2,N))
v_b = sp.simplify(time_derivative(x_b,N))

# Angular Velocity Vector
w_w1 = trans(0, theta_w1.diff(), 0)
w_w2 = trans(0, theta_w2.diff(), 0)
w_b = trans(0, theta_P.diff(), 0)


# Generalized Coordinates
q = sp.Matrix([[theta_w1], [theta_w2], [theta_P]])
qd = q.diff()
qdd = qd.diff()

# Kinetic Energy
T_trans = 0.5*(m_w*v_w1.dot(v_w1) + m_w*v_w2.dot(v_w2) + m_b*v_b.dot(v_b))


T_body = I_b* w_b[1]**2 
T_wheel1 =I_w* w_w1[1]**2 
T_wheel2 =I_w* w_w2[1]**2 

T_rot = 0.5*(T_body + T_wheel1  +  T_wheel2)

# Potential Energy
V =  m_b*g*(l*sp.cos(theta_P)) + 2*m_w*g*r 

# Lagrangian
L = T_trans + T_rot - V


In [4]:
T_trans

0.5*m_b*(l**2*sin(theta_P(t))**2*Derivative(theta_P(t), t)**2 + (l*cos(theta_P(t))*Derivative(theta_P(t), t) + 0.5*r*Derivative(theta_w1(t), t) + 0.5*r*Derivative(theta_w2(t), t))**2) + 0.5*m_w*r**2*Derivative(theta_w1(t), t)**2 + 0.5*m_w*r**2*Derivative(theta_w2(t), t)**2

In [5]:
tau = get_torque_from_L(L,q,qd)
tau = sp.simplify(tau)
# Ml, Cl, Gl, Wl = ir.get_EoM_from_T(tau,qdd,g)
# Ml, Cl, Gl, Wl

In [6]:
tau

Matrix([
[1.0*I_w*Derivative(theta_w1(t), (t, 2)) + 0.5*m_b*r*(-l*sin(theta_P(t))*Derivative(theta_P(t), t)**2 + l*cos(theta_P(t))*Derivative(theta_P(t), (t, 2)) + 0.5*r*Derivative(theta_w1(t), (t, 2)) + 0.5*r*Derivative(theta_w2(t), (t, 2))) + 1.0*m_w*r**2*Derivative(theta_w1(t), (t, 2))],
[1.0*I_w*Derivative(theta_w2(t), (t, 2)) + 0.5*m_b*r*(-l*sin(theta_P(t))*Derivative(theta_P(t), t)**2 + l*cos(theta_P(t))*Derivative(theta_P(t), (t, 2)) + 0.5*r*Derivative(theta_w1(t), (t, 2)) + 0.5*r*Derivative(theta_w2(t), (t, 2))) + 1.0*m_w*r**2*Derivative(theta_w2(t), (t, 2))],
[                                         1.0*I_b*Derivative(theta_P(t), (t, 2)) - 1.0*g*l*m_b*sin(theta_P(t)) + 1.0*l**2*m_b*Derivative(theta_P(t), (t, 2)) + 0.5*l*m_b*r*cos(theta_P(t))*Derivative(theta_w1(t), (t, 2)) + 0.5*l*m_b*r*cos(theta_P(t))*Derivative(theta_w2(t), (t, 2))]])

In [7]:
T_w1, T_w2 = sp.symbols('T_w1, T_w2')
u = sp.Matrix([[T_w1], [T_w2]])

u_matrix = sp.Matrix([[-T_w1], [-T_w2], [ T_w1+ T_w2]])

In [8]:
tau_eq = tau+u_matrix

In [9]:
M, C, G, W = get_EoM_from_T(tau_eq,qdd,g, u)

In [10]:
linearlize_eq = {sp.sin(theta_P):theta_P, sp.cos(theta_P):1, theta_P.diff()**2:0}
tau_linear = sp.simplify(tau_eq.subs(linearlize_eq))
tau_linear

Matrix([
[1.0*I_w*Derivative(theta_w1(t), (t, 2)) - T_w1 + 0.5*m_b*r*(l*Derivative(theta_P(t), (t, 2)) + 0.5*r*Derivative(theta_w1(t), (t, 2)) + 0.5*r*Derivative(theta_w2(t), (t, 2))) + 1.0*m_w*r**2*Derivative(theta_w1(t), (t, 2))],
[1.0*I_w*Derivative(theta_w2(t), (t, 2)) - T_w2 + 0.5*m_b*r*(l*Derivative(theta_P(t), (t, 2)) + 0.5*r*Derivative(theta_w1(t), (t, 2)) + 0.5*r*Derivative(theta_w2(t), (t, 2))) + 1.0*m_w*r**2*Derivative(theta_w2(t), (t, 2))],
[     1.0*I_b*Derivative(theta_P(t), (t, 2)) + T_w1 + T_w2 - 1.0*g*l*m_b*theta_P(t) + 1.0*l**2*m_b*Derivative(theta_P(t), (t, 2)) + 0.5*l*m_b*r*Derivative(theta_w1(t), (t, 2)) + 0.5*l*m_b*r*Derivative(theta_w2(t), (t, 2))]])

In [11]:
Ml, Cl, Gl, Wl = get_EoM_from_T(tau_linear,qdd,g, u)

In [23]:
param = {I_w:0.199893505, I_b:2.139168508, m_b:40, m_w:15, r:0.165,l:0.3,  g:9.81}

Mlp = msubs(Ml, param)
Clp = msubs(Cl, param)
Glp = msubs(Gl, param)
Wlp = msubs(Wl, param)

Mlp_inv = Mlp.inv()
qdd_rhs_A = Mlp_inv*(-Clp -Glp)
qdd_rhs_B = Mlp_inv*Wlp*u

X = q.col_join(qd)
Xd_A = qd.col_join(qdd_rhs_A)
Xd_B = qd.col_join(qdd_rhs_B)
U = u

A = Xd_A.jacobian(X)
B = Xd_B.jacobian(U)
C = X.jacobian(X)
D = X.jacobian(U)

ss0 = [A, B, C, D]
sys0 = control.ss(*[pl.array(mat_i.subs(param)).astype(float) for mat_i in ss0])
mprint(sys0)

A = [[  0.           0.           0.           1.           0.
        0.        ]
     [  0.           0.           0.           0.           1.
        0.        ]
     [  0.           0.           0.           0.           0.
        1.        ]
     [  0.           0.         -25.03210719   0.           0.
        0.        ]
     [  0.           0.         -25.03210719   0.           0.
        0.        ]
     [  0.           0.          29.1477018    0.           0.
        0.        ]]

B = [[ 0.          0.        ]
     [ 0.          0.        ]
     [ 0.          0.        ]
     [ 1.65100151  0.00699069]
     [ 0.00699069  1.65100151]
     [-0.46024303 -0.46024303]]

C = [[1. 0. 0. 0. 0. 0.]
     [0. 1. 0. 0. 0. 0.]
     [0. 0. 1. 0. 0. 0.]
     [0. 0. 0. 1. 0. 0.]
     [0. 0. 0. 0. 1. 0.]
     [0. 0. 0. 0. 0. 1.]]

D = [[0. 0.]
     [0. 0.]
     [0. 0.]
     [0. 0.]
     [0. 0.]
     [0. 0.]]

